In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


# ==========================================================
# Sliding-window builder
# ==========================================================
def build_sliding_window(X, y, steps):
    X_seq, y_seq = [], []
    for i in range(len(X) - steps):
        X_seq.append(X[i:i+steps])
        y_seq.append(y[i+steps])
    return np.array(X_seq), np.array(y_seq)


# ==========================================================
# GRU MODEL (PyTorch)
# ==========================================================
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(input_size, hidden_size, batch_first=True)
        self.gru2 = nn.GRU(hidden_size, hidden_size, batch_first=True)

        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)

        x = x[:, -1, :]        # last time step
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


# ==========================================================
# Experiment parameters
# ==========================================================
symbol_list = ['AAPL','NVDA','MSFT','META','GOOG','AMZN','F','TGT','EL']
#symbol_list = ['F','TGT','EL']
steps_list = [2, 5, 10]
seeds = [0, 1, 2, 3]


# ==========================================================
# MAIN LOOP
# ==========================================================
for symbol in symbol_list:

    print(f"\n==================== {symbol} ====================")

    for steps in steps_list:
        print(f"\n---- Window = {steps} ----")

        re_train_error = []
        re_train_acc = []
        re_testing_error = []
        re_testing_acc = []

        for seed in seeds:

            # Set seed
            torch.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            np.random.seed(seed)
            random.seed(seed)

            # ==========================================================
            # Load data
            # ==========================================================
            data = pd.read_csv(f'../../Datasets/data{symbol}.csv')

            data.index = data['Date']
            data.drop(columns=['Date'], inplace=True)

            # remove inf columns
            columns_infinity = data.columns.to_series()[np.isinf(data).any()]
            data.drop(columns=columns_infinity, inplace=True)

            # feature selection
            data = data[['Close','pct_change','log_change',
                         '7sma','14sma','21sma','7ema','14ema','21ema',
                         '7macd','14macd',
                         '7rsi','14rsi','21rsi',
                         '7atr','14atr','21atr',
                         '7upper','7lower',
                         '14upper','14lower',
                         '21upper','21lower',
                         '7rsv','14rsv','21rsv',
                         '3compo_FT','6compo_FT','9compo_FT',
                         '27compo_FT','81compo_FT','100compo_FT']]

            # target = close
            data['y'] = data['Close']
            data.drop(columns=['Close'], inplace=True)

            X_all = data.iloc[:, :-1].values
            y_all = data.iloc[:, -1].values

            # ==========================================================
            # Train/test split BEFORE scaling
            # ==========================================================
            split_idx = int(0.8 * len(X_all))
            X_train_raw, X_test_raw = X_all[:split_idx], X_all[split_idx:]
            y_train_raw, y_test_raw = y_all[:split_idx], y_all[split_idx:]

            # ==========================================================
            # Scaling (fit on training only)
            # ==========================================================
            x_scaler = MinMaxScaler()
            y_scaler = MinMaxScaler()

            X_train = x_scaler.fit_transform(X_train_raw)
            X_test = x_scaler.transform(X_test_raw)

            y_train = y_scaler.fit_transform(y_train_raw.reshape(-1,1)).flatten()
            y_test = y_scaler.transform(y_test_raw.reshape(-1,1)).flatten()

            # ==========================================================
            # Create sliding windows AFTER scaling
            # ==========================================================
            X_train_seq, y_train_seq = build_sliding_window(X_train, y_train, steps)
            X_test_seq, y_test_seq = build_sliding_window(X_test, y_test, steps)

            print("Train:", X_train_seq.shape, y_train_seq.shape)
            print("Test:",  X_test_seq.shape,  y_test_seq.shape)

            # convert to tensors
            trainX = torch.tensor(X_train_seq, dtype=torch.float32)
            trainY = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1)

            testX = torch.tensor(X_test_seq, dtype=torch.float32)
            testY = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1)

            # datasets
            train_loader = DataLoader(TensorDataset(trainX, trainY), batch_size=128, shuffle=True)

            # ==========================================================
            # Model
            # ==========================================================
            input_size = X_train_seq.shape[2]  # number of features
            hidden_size = 128

            model = GRUModel(input_size, hidden_size, output_size=1)

            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001)

            # ==========================================================
            # Train
            # ==========================================================
            epochs = 60
            for epoch in range(epochs):
                model.train()
                total_loss = 0
                for x_batch, y_batch in train_loader:
                    optimizer.zero_grad()
                    pred = model(x_batch)
                    loss = criterion(pred, y_batch)
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()

                print(f"Epoch {epoch+1}/{epochs} - Loss {total_loss/len(train_loader):.4f}")

            # ==========================================================
            # Predict (train + test)
            # ==========================================================
            model.eval()

            train_pred = model(trainX).detach().numpy()
            test_pred = model(testX).detach().numpy()

            # inverse scale
            train_pred = y_scaler.inverse_transform(train_pred)
            test_pred = y_scaler.inverse_transform(test_pred)

            train_true = y_scaler.inverse_transform(trainY.numpy())
            test_true = y_scaler.inverse_transform(testY.numpy())

            # ==========================================================
            # Metrics
            # ==========================================================
            train_RMSE = np.sqrt(mean_squared_error(train_true, train_pred))
            test_RMSE  = np.sqrt(mean_squared_error(test_true, test_pred))

            train_acc = 1 - np.mean(abs((train_true - train_pred) / train_true))
            test_acc  = 1 - np.mean(abs((test_true - test_pred) / test_true))

            re_train_error.append(train_RMSE)
            re_train_acc.append(train_acc)
            re_testing_error.append(test_RMSE)
            re_testing_acc.append(test_acc)

        # ==========================================================
        # Save result per symbol & window size
        # ==========================================================
        df = pd.DataFrame({
            "Seed": seeds,
            "Train Error": re_train_error,
            "Train Accuracy": re_train_acc,
            "Test Error": re_testing_error,
            "Test Accuracy": re_testing_acc
        })

        df.to_csv(f"GRU_results_{symbol}_{steps}.csv", index=False)
        print(f"Saved → results_{symbol}_GRU_steps{steps}.csv")
